# Figure 5 Notebook
This notebook creates Figure 5 for a selected scenario related to green methanol production systems. (*full abatement*)

## Main Function
Creates Figure 5 for a selected scenario related to green methanol production systems.

## Inputs
- Scenario and year specified in the notebook.
- Data from `results_meoh_case.xlsx` file in the `Input` directory.
- Data from `Ecoinvent_to_PREMISE.xlsx` file in the `Input` directory.
- Data from `countries.geojson` file in the `Input` directory.

## Outputs
- Plot saved as 'Output/Figure 5 [scenario] [year].png'
- Processed data saved as 'Output/Figure 5 [scenario] [year].xlsx'

The geopandas package is required for this figure

In [ ]:
scenario = 'PkBudg500'
year = '2030'

### Import Section

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import copy
import geopandas
from mpl_toolkits.axes_grid1 import make_axes_locatable
import copy
import os
import pandas as pd

plt.rcParams["figure.figsize"] = [10, 5.00]
plt.rcParams["figure.autolayout"] = True

### Selecting a colormap

In [ ]:
import matplotlib.colors as clr
some_matrix = np.random.rand(10,10)
cmap = clr.LinearSegmentedColormap.from_list('custom green', ['#4492f4','#6b33a3','#f84540'], N=256)

### Prepare data

Mapping Data 🗺️

Import the geography (country border) files via geopandas / geojson.

In [ ]:
countries = geopandas.read_file('Input/countries.geojson')

Combine the countries to the REMIND supra-regions. The "Ecoinvent_to_PREMIS.xslx" contains the mapping of country to region.

In [ ]:
image_regions = countries.copy()
image_regions.set_index(['shortname'],inplace=True)

# Define regions and sheet number
regions = ["LAM", "OAS", "SSA", "EUR", "NEU", "MEA", "REF", "CAZ", "CHA", "IND", "JPN", "USA"]

# Load the Excel file
df  = pd.read_excel('Input/Ecoinvent_to_PREMISE.xlsx', keep_default_na=False)

# Create a dictionary of regions
regions_dict = {reg: [] for reg in df}
for reg in df:
    for i in df.index:
        if df.loc[i, reg] != 0:
            regions_dict[reg].append(df.loc[i, reg])

# Create premise_regions DataFrame
premise_regions = {
    reg: image_regions.loc[regions_dict[reg]]['geometry'].unary_union
    for reg in regions
}

premise_regions = pd.DataFrame({"geometry": premise_regions})
premise_regions = geopandas.GeoDataFrame(premise_regions, geometry='geometry')

Production data 🧪

Import the results from the optimization. The "results_meoh_case.xlsx" file contains the solution to several discrete values of epsilon. Thus, to find the solution which just abates for the emissions of BAU methanol production, a linear interpolation between the two adjacent values is performed:

In [ ]:
df = pd.read_excel('Input/results_meoh_case.xlsx')

In [ ]:
# reduce file to the selected scenario / year
filtered_df = df[(df['scenario'] == scenario) & (df['year'] == int(year))]
# find all the "BAU" rows, there the GWP reduction data is storred
filtered_df2 = filtered_df[filtered_df['net-zero GWP [Mt]'] != 0]
# read the gwp reduction target
net_zero_gwp = max(filtered_df2['net-zero GWP [Mt]'])
# filter the data to those there the reduction potential is smaller or bigger than the target
filtered_df3 = filtered_df2[filtered_df2['GWP reduction [Mt]'] < net_zero_gwp]
filtered_df4 = filtered_df2[filtered_df2['GWP reduction [Mt]'] > net_zero_gwp]
# read the closest values to the target
lower = max(filtered_df3['GWP reduction [Mt]'])
upper = min(filtered_df4['GWP reduction [Mt]'])
# get the associated epsilon values
lower_eps = max(filtered_df3['epsilon [%]'])
upper_eps = min(filtered_df4['epsilon [%]'])
# find the epsilon which linearily interpolates between the two closest solutions and the target
eps = (net_zero_gwp - lower)/(upper - lower)*(upper_eps-lower_eps) + lower_eps
# Use this epsilon to interpolate all the production data results

In [ ]:
df1 = filtered_df[filtered_df['epsilon [%]'] == lower_eps]
df2 = filtered_df[filtered_df['epsilon [%]'] == upper_eps]

In [ ]:
# Create a new DataFrame for interpolated data
interpolated_rows = []

# Iterate through each row and interpolate the values based on epsilon
for index, row1 in df1.iterrows():
    df2red = df2[(df2['region'] == row1['region']) & (df2['tech'] == row1['tech'])]
    for index2, row2 in df2red.iterrows():
        if row1['epsilon [%]'] == eps:
            # No interpolation needed, just append the row from df1
            interpolated_rows.append(row1)
        else:
            # Linear interpolation
            alpha = (eps - lower_eps) / (upper_eps - lower_eps)
            interpolated_values = {}
            for column in df1.columns:
                if column not in ['scenario', 'year', 'epsilon [%]', 'region', 'tech']:
                    interpolated_values[column] = alpha * row2[column] + (1 - alpha) * row1[column]
                elif column == 'epsilon [%]':
                    interpolated_values[column] = eps
                else:
                    interpolated_values[column] = row1[column]
            interpolated_rows.append(interpolated_values)

# Create the interpolated DataFrame using pandas.concat
interpolated_df = pd.concat([pd.DataFrame([row]) for row in interpolated_rows], ignore_index=True)

prod_data = interpolated_df

### Plotting function 🎨

Coordinates to properly align the boxes with the regions

In [ ]:
coords = {'CAZ': [-110, 60],
          'CHA': [97,30],
          'EUR': [10,50],
          'IND': [80,10],
          'JPN': [160,20],
          'LAM': [-60,-10],
          'MEA': [35,25],
          'NEU': [-20,65],
          'OAS': [105, 51],
          'REF': [65,60],
          'SSA': [25,0],
          'USA': [-100,40]}

(Complex) plotting function

In [ ]:
def plot_CO2_map(premise_regions, data):
    bau_production = float(data[data['tech'] == 'BAU']['meoh [Mt]'])
    
    premise_regions['choice'] = 0.0
    premise_regions['choice2'] = 0.0
    premise_regions['choice3'] = 0.0
    premise_regions['choice4'] = 0.0
    
    idx_old = 0
    for ind in data.index:
        tech = data['tech'][ind]
        reg = data['region'][ind]
        if 'DAC' == tech:
            premise_regions['choice'][reg] = data['meoh [Mt]'][ind]
        if 'PSC' == tech:
            premise_regions['choice2'][reg] = data['meoh [Mt]'][ind]
    
    for reg in premise_regions.index:
        premise_regions['choice4'][reg] = premise_regions['choice2'][reg]+premise_regions['choice'][reg]
        if premise_regions['choice2'][reg] != 0 and premise_regions['choice'][reg] is not 0:
            premise_regions['choice3'][reg] = premise_regions['choice'][reg]/(premise_regions['choice2'][reg]+premise_regions['choice'][reg])
        if premise_regions['choice2'][reg] == 0:
            premise_regions['choice3'][reg] = 1
        if premise_regions['choice'][reg] <= 0.0001:
            premise_regions['choice3'][reg] = 0.001
        
    plt.rcParams['figure.figsize'] = [7.08661, 5]

    # Map for Methanol from DAC CO2
    fig, axs = plt.subplots(1, 1)
    divider = make_axes_locatable(axs)

    cax = divider.append_axes("right", size="5%", pad=0.1)    
    premise_regions.plot(column='choice3', ax=axs, legend=False, edgecolor='k', cmap=cmap, facecolor="none", vmin=0, vmax=5) #, legend_kwds={'label': 'Methanol from DAC CO2 [MT]', "orientation": "vertical"})
    #premise_regions.plot(column='choice', ax=axs, legend=True, cax=cax, edgecolor='k', cmap='OrRd', facecolor="none", vmin=0, legend_kwds={'label': 'Methanol from DAC CO2 [MT]', "orientation": "vertical"})
    premise_regions1 = premise_regions[premise_regions['choice3'] > 0]
    premise_regions1.plot(column='choice3', ax=axs, legend=False, edgecolor='k', cmap=cmap, vmin=0, vmax=1) #, legend_kwds={"orientation": "vertical", "ticks": [0,0.25,0.5,0.75,1]})
    #premise_regions1.plot(column='choice', ax=axs, legend=True, cax=cax, edgecolor='k', cmap='OrRd', vmin=0, legend_kwds={'label': 'Methanol from DAC CO2 [MT]', "orientation": "vertical"})
    
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=0, vmax=1))
    sm._A = []
    cbr = fig.colorbar(sm, cax=cax, ticks=[0,0.5,1], orientation="vertical")
    cbr.ax.set_yticklabels(['PSC', 'DAC & PSC','DAC'])
    #cbr.ax.tick_params(rotation=90)
    
    'CAZ'
    axs.annotate("", xy=(-110, 58), xytext=(-90, 51.36), arrowprops={"color": "black", "arrowstyle": "-", "linestyle": "--"})
    axs.annotate("", xy=(125, -20.02), xytext=(140, -25), arrowprops={"color": "black", "arrowstyle": "-", "linestyle": "--"})
    circle = plt.Circle((140, -25), 2, edgecolor='black', facecolor='gray', zorder=3, linewidth=1)
    axs.add_patch(circle)
    
    'USA' #-100,40
    axs.annotate("", xy=(-100,40), xytext=(-150,65), arrowprops={"color": "black", "arrowstyle": "-", "linestyle": "--"})
    circle = plt.Circle((-150,65), 2, edgecolor='black', facecolor='gray', zorder=3, linewidth=1)
    axs.add_patch(circle)
    
    'NEU'
    axs.annotate("", xy=(-20, 65), xytext=(35,40), color="black", arrowprops={"color": "black", "arrowstyle": "-", "linestyle": "--"})
    circle = plt.Circle((35,40), 2, edgecolor='black', facecolor='gray', zorder=3, linewidth=1)
    axs.add_patch(circle)
    axs.annotate("", xy=(-20, 65), xytext=(7, 60), arrowprops={"color": "black", "arrowstyle": "-", "linestyle": "--"})
    circle = plt.Circle((7, 60), 2, edgecolor='black', facecolor='gray', zorder=3, linewidth=1)
    axs.add_patch(circle)
    
    'JAP'
    axs.annotate("", xy=(170, 20), xytext=(140,40), color="black", arrowprops={"color": "black", "arrowstyle": "-", "linestyle": "--"})
    circle = plt.Circle((140,40), 2, edgecolor='black', facecolor='gray', zorder=3, linewidth=1)
    axs.add_patch(circle)
    
    'OAS'
    axs.annotate("", xy=(105, 50), xytext=(125,0), color="black", arrowprops={"color": "black", "arrowstyle": "-", "linestyle": "--"})
    circle = plt.Circle((125,0), 2, edgecolor='black', facecolor='gray', zorder=3, linewidth=1)
    axs.add_patch(circle)
    axs.annotate("", xy=(105, 50), xytext=(67,33), arrowprops={"color": "black", "arrowstyle": "-", "linestyle": "--"})
    circle = plt.Circle((67,33), 2, edgecolor='black', facecolor='gray', zorder=3, linewidth=1)
    axs.add_patch(circle)
    
    props = dict(boxstyle='round', facecolor='white', alpha=1.0)
    for reg in premise_regions.index:
        axs.text(coords[reg][0], coords[reg][1], str(round(premise_regions['choice4'][reg],1)), fontsize=12, verticalalignment='center', horizontalalignment='center', bbox=props)
    
    props = dict(boxstyle='round', facecolor='#D2D2D3', alpha=1.0)
    axs.text(-130, -50, "BAU: " + str(round(bau_production,1)) + " Mt", fontsize=12, color="black", verticalalignment='center', horizontalalignment='center', bbox=props)
    
    axs.set_xlim([-180,180])
    axs.set_ylim([-60,85])
    
    axs.set_xticks([])
    axs.set_yticks([])
    
    # CAZ star
    axs.plot([-90,125], [51.36,-20.02], markersize=10, marker='*', markerfacecolor='gray', markeredgecolor='black', linestyle = 'None',zorder=15)
    
    plt.savefig('Output/Figure 5 ' + scenario + '_' + str(year) + '.png', dpi=600)

### Execute plotting ~

In [ ]:
plot_CO2_map(premise_regions, prod_data)

In [ ]:
df = premise_regions[['choice2','choice3','choice4']].rename(
    columns={"choice2": "PSC [-]", "choice3": "DAC [-]", "choice4": "CCU methanol production [Mt]"},
)

In [ ]:
df.to_excel('Output/Figure 5 ' + scenario + '_' + str(year) + '.xlsx')